In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import splitfolders
splitfolders.ratio(r'/content/drive/MyDrive/Colab Notebooks/dataset/Data(melspectrogram)', output=r'/content/C:\Users\USER\Untitled Folder\Data/', seed=1337, ratio=(0.8, 0.1, 0.1))
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.applications.resnet import ResNet50 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical
batch_size=25
test_batch_size=32 
test_steps=1
train_path = r'/content/C:\Users\USER\Untitled Folder\Data/train'
test_path = r'/content/C:\Users\USER\Untitled Folder\Data/test'
val_path = r'/content/C:\Users\USER\Untitled Folder\Data/val'
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps)
def scalar(img):    
    return img  # EfficientNet expects pixelsin range 0 to 255 so no scaling is required
trgen=ImageDataGenerator(preprocessing_function=scalar, horizontal_flip=True)
tvgen=ImageDataGenerator(preprocessing_function=scalar)
train_generator=trgen.flow_from_directory( directory=train_path , target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)
test_generator=tvgen.flow_from_directory( directory=test_path, target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=False, batch_size=test_batch_size)

valid_generator=tvgen.flow_from_directory( directory=val_path, target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)
classes=list(train_generator.class_indices.keys())
class_count=len(classes)
train_steps=int(np.ceil(len(train_generator.labels)/batch_size))




Copying files: 529 files [06:32,  1.35 files/s]


test batch size:  32   test steps:  1
Found 411 images belonging to 28 classes.
Found 78 images belonging to 28 classes.
Found 40 images belonging to 28 classes.


In [4]:
lrr= ReduceLROnPlateau(monitor='val_acc', factor=.01, patience=3, min_lr=1e-5)
base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3), classes = 28)


94781440/94765736 [==============================] - 1s 0us/step


In [5]:
import tensorflow as tf

In [6]:
model= Sequential()
model.add(base_model) 
model.add(Flatten()) 
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
#model.add(Dropout(.2))
model.add(Dense(28,activation=('softmax'))) 

#Checking the final model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               3

In [7]:
learn_rate=.001
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.fit(train_generator,epochs = 10, validation_data = valid_generator, verbose = 1)

Epoch 1/10
17/17 [==============================] - 25s 441ms/step - loss: 3.1665 - accuracy: 0.1533 - val_loss: 2.7090 - val_accuracy: 0.2750
Epoch 2/10
17/17 [==============================] - 5s 279ms/step - loss: 2.2439 - accuracy: 0.3966 - val_loss: 2.1810 - val_accuracy: 0.4500
Epoch 3/10
17/17 [==============================] - 5s 280ms/step - loss: 1.7544 - accuracy: 0.5158 - val_loss: 2.3090 - val_accuracy: 0.2750
Epoch 4/10
17/17 [==============================] - 5s 281ms/step - loss: 1.3754 - accuracy: 0.5839 - val_loss: 1.9887 - val_accuracy: 0.4250
Epoch 5/10
17/17 [==============================] - 5s 290ms/step - loss: 1.0536 - accuracy: 0.6667 - val_loss: 1.3708 - val_accuracy: 0.5500
Epoch 6/10
17/17 [==============================] - 5s 282ms/step - loss: 0.7223 - accuracy: 0.7956 - val_loss: 1.5294 - val_accuracy: 0.6000
Epoch 7/10
17/17 [==============================] - 5s 284ms/step - loss: 0.5801 - accuracy: 0.8345 - val_loss: 1.3798 - val_accuracy: 0.6500
Epoch

In [9]:
model.evaluate(test_generator)

3/3 [==============================] - 2s 337ms/step - loss: 2.1264 - accuracy: 0.6154


[2.1264326572418213, 0.6153846383094788]